# AFSK Demodulator
## Step 5: Hardware I/O Interlude

-----

In this notebook we take a brief break from the AFSK demodulator project to play around with the I/O features of the Pynq board.  We will be making use of these features in the coming segments.  This is a good time to learn how to use the buttons and switches -- and LEDs -- on the Pynq board.

This code is part of the [AFSK Demodulator on Pynq](afsk-demodulator-fpga.ipynb) project.  However, this section is completely stand-alone.  It does not depend on any of the previous work like most of the other sections.

The purpose of this code is to help further our migration of the Python demodulator code to FPGA.  In the next section we are going to implement the Digital PLL.  When the PLL is locked, we have detected a data carrier.  We will use this lock indicator to signal "data carrier detection" by turning on an LED on when the PLL is locked, and turning it off when it is unlocked.  To do that, we need to first learn how to interact with the digital I/O interfaces on the Pynq board.

We are going to need a file from the TUL web side called a device constraints file (XDC).

http://www.tul.com.tw/download/pynq-z2_v1.0.xdc.zip

If you are not using a Pynq Z2, you will need to get the proper XDC file for your board.

## Prerequisites

At this point you are expected to have:

 * A configured PYNQ environment.
 * Vivado installed on your computer and configured for your board.
 * Experience working through the tutorials at https://pynq.readthedocs.io/.

You do not really need to have done anything further with the demodulator project.  This is a fairly stand-alone project.


## Outline

We are goint to learn to interact with some of the I/O components which are attached to the PL of the Pynq board.  We are going to use the push buttons and the 4 green LEDs.  We are also going to trigger an interrupt on the PS and use the asynchronous I/O features of Python and Pynq.

We are going to create PL code that implements a 4-bit counter, displaying the binary values in the green LEDs on the Pynq board, and using the push buttons near the LEDs for input.  We will have an Up/Down button pair, a Reset to Zero button, and a Data Ready button which we will use to indicate to the PS that a value is ready to be read.

We will perform the following steps in this section:

 1. Create a C++ file that reads the buttons as input, writes to the LEDs as output, has an AXI interface to the counter, and a return value that indicates to the PS to read the counter.
 1. Create a C++ test case for the above file.
 1. Generate an IP package from the code that can be used in Vivado.
 1. Create a Zynq project in Vivado that uses the IP.
 1. Import the XDC file and modify it to expose the LEDs and buttons we will be using.
 1. Export the bitstream for our project from Vivado.
 1. Use Python running on the PS to load the bitstream to the PL, and verify that it works.


## Vivado HLS

The code we are going to write will be fairly simple. However, there are a few things we have to do that may not be obvious, and there are a few things I learned about HLS during this project.

Since we are going to be reading button presses, and the buttons we will be using are not debounced in hardware, we need to implement a debounce function for our keys.  Mechanical switches are noisy.  A switch does not make a clean jump from OFF to ON.  At high resolution, there are a lot of on/off transitions that occur during a button press.  We need to ignore those.

Debounce is implemented by having a key press trigger a counter that waits a short time before reading the key value.  This causes all of the make/break noise to be ignored.  In an FPGA, this is partly implemented using a state machine.  The states are "waiting for a key press", "waiting for debounce", and "debounce complete". The "debounce complete" state has a very quick transition back to "waiting for key press".

We could debounce each key separately, but there is no need for that and it wastes PL resources.  We can debounce all of the keys at once.  Luckily, this is easy for us to do.

Efficient implementation of this code is tricky.  My first attempt at it was estimated to take well over 1000 LUTS.  That's a lot for something as simple as this.  Efficient implementation of the debounce state machine and of the main loop takes a bit of work.  The end result uses 31/42 LUTs/FFs after Vivado synthesis, although the HLS synthesis step estimated it would use 264/98.  The estimated number of LUTs was off by an order of magnitude.

 1. Start Vivado HLS.
    ```bash
    vivado_hls
    ```
 1. Create a new project under the project_07 directory call HLS.
 1. Create a top-level function called count.
 1. Create 2 new files:
    * [count.hpp](HLS/count.hpp)
    * [count.cpp](HLS/count.cpp)
 1. Create a new test bench:
    * [count_test.cpp](HLS/count_test.cpp)
 

-----

This is the header, which countains the debounce code:

```c++
#include <ap_int.h>

typedef ap_uint<4> leds_type;
typedef ap_uint<4> btns_type;

ap_uint<1> count(volatile leds_type& leds_out, int& cntr, btns_type btn_in);

class Debouncer {
        ap_uint<17> counter;
public:
        void start() {counter = 0;}
        void inc() {if (counter[16] != 1) ++counter;}
        bool ok() const {return counter[16] == 1;}
};

class Button {
        enum State { UP, DEBOUNCE, DOWN };
        Debouncer debounce_;
        State state_;
public:
        Button() : state_(UP) {}

        template <typename T>
        T state(T button) {

                T result = 0;

                switch (state_) {
                case UP:
                        if (button) {
                                debounce_.start();
                                state_ = DEBOUNCE;
                        }
                        break;
                case DEBOUNCE:
                        debounce_.inc();
                        if (debounce_.ok()) {
                                if (button) {
                                        state_ = DOWN;
                                        result = button;
                                } else {
                                        state_ = UP;
                                }
                        }
                        break;
                case DOWN:
                        if (!button) {
                                debounce_.start();
                                state_ = DEBOUNCE;
                        }
                }
                return result;
        }
};

```

And this is the source:

```c++
#include "count.hpp"

ap_uint<1> count(volatile leds_type& leds_out, int& cntr, btns_type btn_in)
{
#pragma HLS INTERFACE ap_none port=leds_out name=led
#pragma HLS INTERFACE s_axilite port=cntr
#pragma HLS INTERFACE ap_none port=btn_in name=btn
#pragma HLS INTERFACE ap_ctrl_hs register port=return

        static ap_uint<4> counter = 8;
        static auto button = Button();

#pragma HLS PIPELINE
        leds_out = counter;

        auto btn = button.state(btn_in);

        if (btn[0] != 0) {
                if (!(counter[0] & counter[1] & counter[2] & counter[3])) counter += 1;
        }
        if (btn[1] != 0) {
                if (!(counter[0] | counter[1] | counter[2] | counter[3])) counter -= 1;
        }
        if (btn[2] != 0) {
                counter = 0;
        }
        cntr = counter;
        return static_cast<ap_uint<1>>(btn[3] != 0);
}
```

Note that the `leds_out` and `btn_in` parameters are set to have an interface of `ap_none`.  These are implemented as bare wires to the input and output hardware.  The `cntr` parameter is exposed as an axilite slave.  We are going to use a return value (the state of button 3) to pass a control signal back to the PS.

Also note that we pass the buttons to the debouncer state machine.  This happens at every clock tick while the block is running (ap_start is asserted).

We also need to declare the return port as ap_ctrl_hs to keep the C/RTL co-simulation system happy.  This will add a bit of a complication to the Vivado block design.

-----

Once the code and test bench are written, we need to run the C simulation, C synthesis, C/RTL co-simulation, then package the IP.  The two simulation steps run our test bench.  This verifies that the code will sythesize properly and that it functions properly.

Once the IP is packaged, we are done in HLS.

## Vivado

We will now switch over to Vivado and create a block design. If you have not done so already, unzip the archive containing the XDC constraints file.


 1. Start Vivado and create a new project.
 1. Give it a path -- in our case `afsk-demodulator-pynq/io_interlude` and the name `Vivado`.
 1. Select the `RTL Project` project type.
 1. In the "Default Part" screen, switch to the "Boards" tab. Select the your board from the list.
 1. Click "Finish".
 
We now need to copy the XDC file into the io_interlude/Vivado directory.

With the new project open in Vivado, we need to create a block design.  We are going to follow the some procedure we did in the other projects.

 1. On the right side, in the Flow Navigator, select *Create Block Diagram*.
 1. Use the default name, design_1.
 1. Go into Tools|Settings.
    1. In the settings dialog, choose IP|Repository.
    1. Select "+" to add a repository.
    1. Add io_interlude/HLS as a repository.  You should see that it has 1 IP called `count` in there.
    1. When done, click "OK".
 1. In the Diagram view (main window) select "+" to add IP.
 1. Add the Zynq processing system and run block automation.
 1. When done, double-click the Zynq block and click on "Interrupts" in Page Navigator.
    1. Enable the IRQ_F2P interrupt under Fabric Interrupts|PL-PS Interrupt Ports.
 1. Add an `AXI Interrupt Controller`. Run connection automation.
 1. Edit the `AXI Interrupt Controller`, changing the "Interrupt Output Connection" from "Bus" to "Single".
 1. Connect the `irq` port of the interrupt controller to the `irq_f2p` port on the`Zynq7 Processing System`.
 1. Add the `Count` IP.  Run connection automation.
 1. Connect the `ap_return` port of the `Count` IP block to the AXI interrupt controller's `intr` port.
 1. Add a `Constant` IP. It will have a default value of 1, which is what we want.
 1. Expand the `ap_ctrl` interface on the `Count` IP block.  Attach the Constant block's `dout` to `ap_start`.  This is needed in order to keep the PL running all the time, outside the control of the PS.  Collapse the `ap_ctrl` interface.
 1. In the "Sources" box, click the "+" icon to add a source file.
    1. Select "Add or create contraints" and click "Next".
    1. Click the "+" icon and select "Add files..."
    1. Select the `pynq-z2_v1.0.xdc` file and click "OK".
    1. Click "Finish."
 1. In the "Sources" box, find the new constraints file that was added.
    1. Click on the constraints file to open it in the editor.
    1. Uncomment out the buttons and leds.

    ```tcl
    ##LEDs

    set_property -dict { PACKAGE_PIN R14   IOSTANDARD LVCMOS33 } [get_ports { led[0] }]; #IO_L6N_T0_VREF_34 Sch=led[0]
    set_property -dict { PACKAGE_PIN P14   IOSTANDARD LVCMOS33 } [get_ports { led[1] }]; #IO_L6P_T0_34 Sch=led[1]
    set_property -dict { PACKAGE_PIN N16   IOSTANDARD LVCMOS33 } [get_ports { led[2] }]; #IO_L21N_T3_DQS_AD14N_35 Sch=led[2]
    set_property -dict { PACKAGE_PIN M14   IOSTANDARD LVCMOS33 } [get_ports { led[3] }]; #IO_L23P_T3_35 Sch=led[3]

    ##Buttons

    set_property -dict { PACKAGE_PIN D19   IOSTANDARD LVCMOS33 } [get_ports { btn[0] }]; #IO_L4P_T0_35 Sch=btn[0]
    set_property -dict { PACKAGE_PIN D20   IOSTANDARD LVCMOS33 } [get_ports { btn[1] }]; #IO_L4N_T0_35 Sch=btn[1]
    set_property -dict { PACKAGE_PIN L20   IOSTANDARD LVCMOS33 } [get_ports { btn[2] }]; #IO_L9N_T1_DQS_AD3N_35 Sch=btn[2]
    set_property -dict { PACKAGE_PIN L19   IOSTANDARD LVCMOS33 } [get_ports { btn[3] }]; #IO_L9P_T1_DQS_AD3P_35 Sch=btn[3]
    ```
    
 1. Right click on the `btn_V` port of the Count IP block and select "Make External".  Rename the port to `btn` (it must match the name in the constraints file).
 1. Right click on the `led_V `port of the Count IP block and select "Make External".  Rename the port to `led` so that it matches the constraints file.
 1. Rename the Count block to "count".
 1. Generate the HDL wrapper by clicking on the design in the Sources box, right clicking, and selecting "Generate HDL Wrapper".
![BlockDesign](BlockDesign.png)
 1. Generate the bitstream. Again, this will take some time.
 1. When the bitstream is generated, we need to look at the implemenation to verify that the buttons and LEDs were connected.
    1. Select "Open Implemented Design" in the Flow Navigator on the left panel.
    1. Select "Schematic"
    ![Schematic](Schematic.png)
    1. Verify that the buttons and LEDs are connected as shown in the schematic.
    1. Find the label at the top "138 1/O Ports" and click on it.  A report will open below the schematic diagram.
    1. Find the `led` and `btn` entries in the report and verify that they are assigned to package pins and that they match the entries in the constraints file above.
 1. Go back to the Block Design view. Export the block design (File|Export|Export Block Design...)
 1. Collect the following files:
    - Vivado.srcs/sources_1/bd/design_1/hw_handoff/design_1.hwh
    - Vivado.runs/impl_1/design_1_wrapper.bit
    - design_1.tcl
    * rename these file to "count.{ext}" so that you have count.bit, count.tcl and count.hwh
    ```bash
    mv design_1.tcl count.tcl
    cp ./Vivado.runs/impl_1/design_1_wrapper.bit ./count.bit
    cp ./Vivado.srcs/sources_1/bd/design_1/hw_handoff/design_1.hwh count.hwh
    ```
 1.  Copy these files to `afsk-demodulator-pynq/io_interlude/Pynq/` on the mounted Pynq filesystem.
    ```bash
    cp count.* /var/run/media/${USER}/PYNQ/jupyter_notebooks/afsk-demodulator-pynq/io_interlude/Pynq/

    ```
 1. You can now jump to the Jupyter notebook on the Pynq device.